In [1]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import numpy as np
import tensorflow.keras as keras
from sklearn.metrics import confusion_matrix, f1_score

# Set the path to your dataset
train_dir = '/data/dataset'
test_dir = '/data/dataset'

# Set the image size and batch size
img_size = (299, 299)
batch_size = 32

# Create a data generator for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Add a new classification layer
x = base_model.output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)

predictions = tf.keras.layers.Dense(6, activation='softmax')(x)

# Combine the base model and classification layer
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model's layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Evaluate the model on the test set
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

# Save the model
model.save('/code/model/ResNet50.h5')
labels = ['Foi_Thong', 'Hang_Kra_Rog_Phu_Phan_ST1', 'Hang_Suea_Sakonnakhon_TT1', 'Kroeng_Krawia', 'Tanao_Si_Kan_Khaw_WA1', 'Tanao_Si_Kan_Sang_RD1']

# Train the model and record validation loss and accuracy after each epoch
history = model.fit(train_generator, epochs=10, validation_data=test_generator)

test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

print("\n")

# Generate predictions for the test set
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

# Calculate the confusion matrix and print it
confusion_matrix = confusion_matrix(test_generator.classes, y_pred)
print("Confusion Matrix:\n",confusion_matrix)

num_classes = len(labels)
# Calculate TP, FP, TN, and FN for each class
for i in range(num_classes):
    TP = confusion_matrix[i, i]
    FP = tf.reduce_sum(confusion_matrix[:, i]) - TP
    FN = tf.reduce_sum(confusion_matrix[i, :]) - TP
    TN = tf.reduce_sum(confusion_matrix) - TP - FP - FN
    print(f"\nClass {i} ({labels[i]}):")
    print(f"TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}")


print("\n")
# Calculate precision, recall, and f1-score
precision = np.diag(confusion_matrix) / np.sum(confusion_matrix, axis=0)
recall = np.diag(confusion_matrix) / np.sum(confusion_matrix, axis=1)
f1 = 2 * precision * recall / (precision + recall)

# Print precision, recall, and f1-score
for i, label in enumerate(labels):
    print(f"Label: {label}")
    print(f"Precision: {precision[i]:.3f}")
    print(f"Recall: {recall[i]:.3f}")
    print(f"F1-score: {f1[i]:.3}")

print("\n")
print("RestNet50")

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
print('Accuracy:', test_acc)
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.xticks(range(len(history.history['accuracy'])))
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
print('Loss:', test_loss)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.xticks(range(len(history.history['loss'])))
plt.show()

# Get the weights and biases of the model
weights= model.get_weights()

# Print the shape of the weights and biases
print("Weights shape:", np.shape(weights))
# print("Biases shape:", np.shape(biases))

2023-04-25 09:29:45.736763: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-25 09:29:45.785706: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-25 09:29:45.786511: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 09:29:46.675908: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Found 1832 images belonging to 6 classes.
94765736/94765736 [==============================] - 0s 0us/step
Found 1832 images belonging to 6 classes.


2023-04-25 09:29:50.854573: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


58/58 [==============================] - 70s 1s/step - loss: 2.6826 - accuracy: 0.1894
Test accuracy: 0.1894104778766632
Epoch 1/10


2023-04-25 09:31:04.089945: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


...

2023-04-25 10:00:01.569037: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


58/58 [==============================] - 68s 1s/step - loss: 0.5944 - accuracy: 0.7544
Test accuracy: 0.7543668150901794
Test loss: 0.5944002866744995




2023-04-25 10:01:09.391176: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


58/58 [==============================] - 68s 1s/step
Confusion Matrix:
 [[ 24  73 137  42  27  43]
 [ 38  80 138  33  16  42]
 [ 29  81 115  49  13  39]
 [ 24  61  82  28  18  26]
 [ 27  66 126  48  20  39]
 [ 24  49  91  34  13  37]]

Class 0 (Foi_Thong):
TP: 24, FP: 142, TN: 1344, FN: 322

Class 1 (Hang_Kra_Rog_Phu_Phan_ST1):
TP: 80, FP: 330, TN: 1155, FN: 267

Class 2 (Hang_Suea_Sakonnakhon_TT1):
TP: 115, FP: 574, TN: 932, FN: 211

Class 3 (Kroeng_Krawia):
TP: 28, FP: 206, TN: 1387, FN: 211

Class 4 (Tanao_Si_Kan_Khaw_WA1):
TP: 20, FP: 87, TN: 1419, FN: 306

Class 5 (Tanao_Si_Kan_Sang_RD1):
TP: 37, FP: 189, TN: 1395, FN: 211


Label: Foi_Thong
Precision: 0.145
Recall: 0.069
F1-score: 0.0938
Label: Hang_Kra_Rog_Phu_Phan_ST1
Precision: 0.195
Recall: 0.231
F1-score: 0.211
Label: Hang_Suea_Sakonnakhon_TT1
Precision: 0.167
Recall: 0.353
F1-score: 0.227
Label: Kroeng_Krawia
Precision: 0.120
Recall: 0.117
F1-score: 0.118
Label: Tanao_Si_Kan_Khaw_WA1
Precision: 0.187
Recall: 0.061
F1-score:

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (322,) + inhomogeneous part.

In [2]:
# Use the model to make predictions on new images
import matplotlib.pyplot as plt
import tensorflow.keras as keras

#figure 1
new_img = tf.keras.preprocessing.image.load_img('/data/test/test1.jpg', target_size=img_size)

plt.imshow(new_img)
plt.show()

new_img = tf.keras.preprocessing.image.img_to_array(new_img)
#print(new_img)
new_img = new_img / 255.0
new_img = tf.expand_dims(new_img, 0)

prediction = model.predict(new_img)
class_index = tf.argmax(prediction, axis=1)

print('\n')
predicted_label = labels[class_index[0]]
print('Predicted label:', predicted_label)

prediction_values = {}
for i, label in enumerate(labels):
    prediction_values[label] = round(prediction[0][i], 2)

print('Prediction values:', prediction_values)


1/1 [==============================] - 1s 972ms/step


Predicted label: Hang_Kra_Rog_Phu_Phan_ST1
Prediction values: {'Foi_Thong': 0.0, 'Hang_Kra_Rog_Phu_Phan_ST1': 0.84, 'Hang_Suea_Sakonnakhon_TT1': 0.0, 'Kroeng_Krawia': 0.0, 'Tanao_Si_Kan_Khaw_WA1': 0.14, 'Tanao_Si_Kan_Sang_RD1': 0.0}


In [3]:
print("Model Structure : \n")
model.summary()

Model Structure : 

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 305, 305, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 150, 150, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                          

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 75, 75, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 75, 75, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 75, 75, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 75, 75, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 38, 38, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 38, 38, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 38, 38, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 38, 38, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 19, 19, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 19, 19, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 19, 19, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 19, 19, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization) 

 conv4_block5_1_conv (Conv2D)   (None, 19, 19, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 19, 19, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 19, 19, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 19, 19, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_blo

                                                                                                  
 conv5_block1_add (Add)         (None, 10, 10, 2048  0           ['conv5_block1_0_bn[0][0]',      
                                )                                 'conv5_block1_3_bn[0][0]']      
                                                                                                  
 conv5_block1_out (Activation)  (None, 10, 10, 2048  0           ['conv5_block1_add[0][0]']       
                                )                                                                 
                                                                                                  
 conv5_block2_1_conv (Conv2D)   (None, 10, 10, 512)  1049088     ['conv5_block1_out[0][0]']       
                                                                                                  
 conv5_block2_1_bn (BatchNormal  (None, 10, 10, 512)  2048       ['conv5_block2_1_conv[0][0]']    
 ization) 